In [1]:
print("📊 REIMAGE-AI SMART PARKING - MONITORING & DASHBOARD")

# =============================================================================
# ENVIRONMENT CHECK & LAKEHOUSE CONTEXT
# =============================================================================
print("🔧 INITIALIZING ENVIRONMENT...")

try:
    spark  # check if Spark session exists
except NameError:
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.appName("ReimageAI-Monitoring").getOrCreate()
    print("✅ SparkSession created.")

# Attach lakehouse if not already attached (Databricks/Microsoft Fabric-style)
try:
    lakehouses = spark.catalog.listDatabases()
    if not lakehouses:
        print("⚠️ No Lakehouse attached! Please attach the same Lakehouse used in Notebook 2.")
    else:
        active_lakehouse = lakehouses[0].name
        print(f"✅ Using Lakehouse: {active_lakehouse}")
except Exception as e:
    print(f"⚠️ Could not verify lakehouse attachment: {e}")

# Helper to safely run queries
def safe_query(label, query):
    try:
        return spark.sql(query)
    except Exception as err:
        print(f"❌ {label}: {err}")
        return None

# =============================================================================
# STEP 1: System Overview
# =============================================================================
print("\n🔍 STEP 1: SYSTEM OVERVIEW")

try:
    tables = spark.sql("SHOW TABLES").collect()
    print(f"📋 Found {len(tables)} tables in Lakehouse")
    for table in tables:
        tname = table['tableName']
        try:
            cnt = spark.sql(f"SELECT COUNT(*) as cnt FROM {tname}").collect()[0]['cnt']
            print(f"   ✅ {tname}: {cnt} records")
        except:
            print(f"   ⚠️ {tname}: Could not count rows")
except Exception as e:
    print(f"❌ Error listing tables: {e}")

# =============================================================================
# STEP 2: Real-time Analytics
# =============================================================================
print("\n📈 STEP 2: REAL-TIME ANALYTICS")

# Parking Occupancy
print("🅿️ PARKING OCCUPANCY ANALYTICS:")
occupancy_stats = safe_query("Parking analytics",
"""
SELECT 
    parking_zone,
    AVG(CASE WHEN occupancy_status = true THEN 1.0 ELSE 0.0 END) as occupancy_rate,
    COUNT(*) as total_readings,
    MAX(timestamp) as latest_reading
FROM ParkingSensorData
GROUP BY parking_zone
ORDER BY occupancy_rate DESC
""")
if occupancy_stats:
    occupancy_stats.show(10)

# Traffic Congestion
print("\n🚦 TRAFFIC CONGESTION ANALYTICS:")
traffic_stats = safe_query("Traffic analytics",
"""
SELECT 
    congestion_level,
    COUNT(*) as record_count,
    AVG(traffic_density) as avg_density,
    AVG(vehicle_count) as avg_vehicles,
    MAX(timestamp) as latest_data
FROM TrafficCameraData
GROUP BY congestion_level
ORDER BY record_count DESC
""")
if traffic_stats:
    traffic_stats.show(10)

# =============================================================================
# STEP 3: AI System Performance
# =============================================================================
print("\n🤖 STEP 3: AI SYSTEM PERFORMANCE")

# YOLO Processing
print("🖼️ YOLO PROCESSING PERFORMANCE:")
yolo_performance = safe_query("YOLO performance",
"""
SELECT 
    processing_status,
    COUNT(*) as processed_count,
    AVG(processing_confidence) as avg_confidence,
    AVG(ABS(original_vehicle_count - yolo_vehicle_count)) as avg_difference
FROM YOLOProcessedData
GROUP BY processing_status
""")
if yolo_performance:
    yolo_performance.show()

# Prediction System
print("\n🔮 PREDICTION SYSTEM PERFORMANCE:")
try:
    pred_perf = spark.sql("""
        SELECT 
            AVG(confidence) as overall_confidence,
            AVG(similar_patterns_used) as patterns_per_prediction,
            COUNT(*) as total_predictions,
            MIN(prediction_time) as first_prediction,
            MAX(prediction_time) as latest_prediction
        FROM TrafficPredictions
    """).collect()[0]
    print(f"   Overall Confidence: {pred_perf['overall_confidence']:.3f}")
    print(f"   Patterns per Prediction: {pred_perf['patterns_per_prediction']:.1f}")
    print(f"   Total Predictions: {pred_perf['total_predictions']}")
    print(f"   Prediction Range: {pred_perf['first_prediction']} to {pred_perf['latest_prediction']}")
except Exception:
    print("🔮 PREDICTION SYSTEM: No data available")

# =============================================================================
# STEP 4: Data Freshness & System Health
# =============================================================================
print("\n⏰ STEP 4: DATA FRESHNESS & SYSTEM HEALTH")

freshness = safe_query("Freshness check",
"""
SELECT 
    (SELECT MAX(timestamp) FROM ParkingSensorData) as latest_parking_data,
    (SELECT MAX(timestamp) FROM TrafficCameraData) as latest_traffic_data,
    CURRENT_TIMESTAMP() as current_time
""")
if freshness:
    row = freshness.collect()[0]
    print(f"📅 Latest Parking Data: {row['latest_parking_data']}")
    print(f"📅 Latest Traffic Data: {row['latest_traffic_data']}")
    print(f"🕒 Current Time: {row['current_time']}")

# Health Summary
print("\n🏥 SYSTEM HEALTH CHECK:")
tables_health = {
    "Parking Data": "ParkingSensorData",
    "Traffic Data": "TrafficCameraData",
    "Historical Data": "HistoricalTraffic",
    "YOLO Processing": "YOLOProcessedData",
    "Predictions": "TrafficPredictions"
}
for label, tbl in tables_health.items():
    try:
        exists = len(spark.sql(f"SHOW TABLES LIKE '{tbl}'").collect()) > 0
        if not exists:
            print(f"   {label}: ❌ UNAVAILABLE")
            continue
        cnt = spark.sql(f"SELECT COUNT(*) as cnt FROM {tbl}").collect()[0]['cnt']
        status = "✅ HEALTHY" if cnt > 0 else "⚠️ EMPTY"
        print(f"   {label}: {status} ({cnt} records)")
    except Exception:
        print(f"   {label}: ❌ UNAVAILABLE")

# =============================================================================
# FINAL SUMMARY
# =============================================================================
print("\n" + "="*60)
print("🎉 REIMAGE-AI SMART PARKING - DASHBOARD SUMMARY")
print("="*60)

quick = safe_query("Quick stats",
"""
SELECT 
    (SELECT COUNT(*) FROM ParkingSensorData) as parking_records,
    (SELECT COUNT(*) FROM TrafficCameraData) as traffic_records,
    (SELECT COUNT(*) FROM HistoricalTraffic) as historical_records,
    (SELECT COALESCE(COUNT(*), 0) FROM YOLOProcessedData) as ai_processed_images,
    (SELECT COALESCE(COUNT(*), 0) FROM TrafficPredictions) as predictions_generated
""")
if quick:
    q = quick.collect()[0]
    print(f"📊 Parking Records: {q['parking_records']:,}")
    print(f"📊 Traffic Records: {q['traffic_records']:,}")
    print(f"📊 Historical Records: {q['historical_records']:,}")
    print(f"🤖 AI Processed Images: {q['ai_processed_images']:,}")
    print(f"🔮 Predictions Generated: {q['predictions_generated']:,}")

print("\n✅ SYSTEM STATUS: OPERATIONAL")
print("🎉 REIMAGE-AI SMART PARKING SYSTEM IS FULLY OPERATIONAL!")


StatementMeta(, 52de8d9b-dd8d-4855-9fc2-98ad473c99bf, 3, Finished, Available, Finished)

📊 REIMAGE-AI SMART PARKING - MONITORING & DASHBOARD
🔧 INITIALIZING ENVIRONMENT...
⚠️ Could not verify lakehouse attachment: 
[INVALID_IDENTIFIER] The identifier Reimage-AI-Smart-Parking is invalid. Please, consider quoting it with back-quotes as `Reimage-AI-Smart-Parking`.(line 1, pos 7)

== SQL ==
Reimage-AI-Smart-Parking.ParkingDataLakehouse.dbo
-------^^^


🔍 STEP 1: SYSTEM OVERVIEW
📋 Found 5 tables in Lakehouse
   ✅ historicaltraffic: 840 records
   ✅ parkingsensordata: 100 records
   ✅ trafficcameradata: 50 records
   ✅ trafficpredictions: 120 records
   ✅ yoloprocesseddata: 20 records

📈 STEP 2: REAL-TIME ANALYTICS
🅿️ PARKING OCCUPANCY ANALYTICS:
+------------+--------------+--------------+--------------------+
|parking_zone|occupancy_rate|total_readings|      latest_reading|
+------------+--------------+--------------+--------------------+
|      ZONE_A|       0.44000|            25|2025-10-09 16:50:...|
|      ZONE_C|       0.42857|            14|2025-09-27 23:37:...|
|      ZO

#### ENHANCED IMPLEMENTATION WITH HEDERA, MCP & POWER BI

In [2]:
# ==============================================
# 📊 FIXED: POWER BI DASHBOARD & ADVANCED ANALYTICS
# ==============================================
print("📊 REIMAGE-AI SMART PARKING - POWER BI DASHBOARD & ADVANCED ANALYTICS")

from datetime import datetime
import random

# Ensure table exists and add missing columns if required
try:
    cols = [f.name for f in spark.table("TrafficPredictions").schema.fields]
    if "mcp_inference_id" not in cols:
        print("⚙️ Adding missing column: mcp_inference_id to TrafficPredictions...")
        df = spark.read.table("TrafficPredictions")
        df = df.withColumn("mcp_inference_id", lit(None).cast("string"))
        df.write.mode("overwrite").saveAsTable("TrafficPredictions")
except Exception as e:
    print(f"⚠️ Could not verify columns: {e}")

class SafePowerBIDataEngine:
    def __init__(self):
        pass
    
    def safe_float(self, val, default=0.0):
        try:
            return float(val) if val is not None else default
        except:
            return default

    def generate_real_time_metrics(self):
        print("🔄 Generating real-time Power BI metrics...")
        metrics, now = [], datetime.now()

        # === Blockchain & MCP Metrics (Fixed) ===
        try:
            # Ensure table has mcp_inference_id
            blockchain_metrics = spark.sql("""
                SELECT 
                    COUNT(*) as total_tx,
                    COUNT(DISTINCT blockchain_tx_id) as unique_tx,
                    AVG(CASE WHEN blockchain_verified = true THEN 1.0 ELSE 0.0 END) as verified_rate
                FROM ParkingSensorData
                WHERE blockchain_tx_id IS NOT NULL
            """).collect()[0]

            if "mcp_inference_id" in [f.name for f in spark.table("TrafficPredictions").schema.fields]:
                mcp_metrics = spark.sql("""
                    SELECT 
                        COUNT(DISTINCT mcp_inference_id) as audited,
                        COUNT(*) as total
                    FROM TrafficPredictions
                    WHERE mcp_inference_id IS NOT NULL
                """).collect()[0]
            else:
                mcp_metrics = {"audited": 0, "total": 0}

            verified_rate = self.safe_float(blockchain_metrics["verified_rate"])
            audit_rate = self.safe_float(mcp_metrics["audited"]) / max(self.safe_float(mcp_metrics["total"]), 1)

            metrics.append({
                "metric_id": "blockchain_verification_rate",
                "metric_name": "Blockchain Verification Rate",
                "metric_value": verified_rate,
                "metric_timestamp": now,
                "category": "Blockchain Security",
                "zone_id": "SYSTEM_WIDE",
                "data_source": "Hedera Network"
            })
            metrics.append({
                "metric_id": "mcp_audit_coverage",
                "metric_name": "MCP Audit Coverage",
                "metric_value": audit_rate,
                "metric_timestamp": now,
                "category": "Model Governance",
                "zone_id": "SYSTEM_WIDE",
                "data_source": "MCP System"
            })
        except Exception as e:
            print(f"❌ Error in blockchain/MCP metrics: {e}")

        # === Business Metrics (Fixed for NoneType) ===
        try:
            result = spark.sql("""
                SELECT AVG(CASE WHEN occupancy_status = true THEN 1.0 ELSE 0.0 END) AS occ
                FROM ParkingSensorData
            """).collect()[0]
            occ = self.safe_float(result["occ"])
            est_rev = occ * 500 * 2.5
            metrics.append({
                "metric_id": "estimated_revenue",
                "metric_name": "Estimated Hourly Revenue",
                "metric_value": est_rev,
                "metric_timestamp": now,
                "category": "Business Intelligence",
                "zone_id": "CITY_WIDE",
                "data_source": "Revenue Analytics"
            })
        except Exception as e:
            print(f"❌ Error in business metrics: {e}")

        return metrics

    def safe_collect(self, query):
        try:
            df = spark.sql(query)
            rows = df.collect()
            return rows if rows else []
        except:
            return []

pbi = SafePowerBIDataEngine()

# === Generate Metrics ===
print("💾 Saving Power BI optimized metrics...")
metrics = pbi.generate_real_time_metrics()
if metrics:
    spark.createDataFrame(metrics).write.mode("overwrite").format("delta").saveAsTable("PowerBI_Metrics")
    print(f"✅ Saved {len(metrics)} Power BI metrics")

# === Advanced Insights (with safety) ===
print("\n🔍 STEP 3: ADVANCED ANALYTICS & INSIGHTS...")
insights = []

def safe_get(data, index, key, default="N/A"):
    try:
        return data[index][key]
    except:
        return default

# Peak Hours Analysis
rows = pbi.safe_collect("""
    SELECT HOUR(timestamp) AS hr, AVG(CASE WHEN occupancy_status THEN 1 ELSE 0 END) AS rate
    FROM ParkingSensorData GROUP BY hr ORDER BY rate DESC LIMIT 3
""")
if rows:
    insights.append({
        "title": "Peak Hours",
        "description": f"Highest parking at {safe_get(rows,0,'hr')}:00 ({safe_get(rows,0,'rate'):.1%})"
    })

# AI System Health
try:
    ai = spark.sql("SELECT AVG(processing_confidence) AS conf, COUNT(*) AS cnt FROM YOLOProcessedData").collect()[0]
    conf = pbi.safe_float(ai["conf"])
    health = "HEALTHY" if conf >= 0.85 else "DEGRADED"
    insights.append({
        "title": "AI System Health",
        "description": f"AI {health}, avg confidence {conf:.1%}"
    })
except Exception as e:
    print(f"❌ Error in AI health: {e}")

print(f"\n💡 ADVANCED INSIGHTS GENERATED: {len(insights)}")
for i in insights:
    print(f"🔍 {i['title']} - {i['description']}")

print("\n✅ FIXED DASHBOARD EXECUTION COMPLETED SUCCESSFULLY")


StatementMeta(, 9997134e-9f4b-4486-89c2-da64b508e186, 4, Finished, Available, Finished)

📊 REIMAGE-AI SMART PARKING - POWER BI DASHBOARD & ADVANCED ANALYTICS
💾 Saving Power BI optimized metrics...
🔄 Generating real-time Power BI metrics...
✅ Saved 3 Power BI metrics

🔍 STEP 3: ADVANCED ANALYTICS & INSIGHTS...

💡 ADVANCED INSIGHTS GENERATED: 2
🔍 Peak Hours - Highest parking at 11:00 (100.0%)
🔍 AI System Health - AI DEGRADED, avg confidence 0.0%

✅ FIXED DASHBOARD EXECUTION COMPLETED SUCCESSFULLY
